In [1]:
from twitterscraper import query_tweets

from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY
from datetime import datetime, date, timedelta

from proxy_utils import proxy_dict, get_proxies

import pandas as pd
import time

import os

import requests

from glob import glob

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

%load_ext autoreload
%autoreload 2

In [2]:
tweets = query_tweets("Bitcoin", limit=10000, begindate=date(2018,4,1), enddate=date(2018, 4, 5))

INFO:root:Querying Bitcoin since:2018-04-01 until:2018-04-02
INFO:root:Querying Bitcoin since:2018-04-03 until:2018-04-04
INFO:root:Querying Bitcoin since:2018-04-02 until:2018-04-03
INFO:root:Querying Bitcoin since:2018-04-04 until:2018-04-05
ERROR:root:Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-980587316925419520-980594113174294528&q=Bitcoin%20since%3A2018-04-01%20until%3A2018-04-02&l=".
Traceback (most recent call last):
  File "/home/shresthanikesh23/jupyter/CryptoTraderRemote/CryptoTrader/data_utils/twitter_data/twitterscraper/query.py", line 54, in query_single_page
    json_resp = json.loads(response.text)
  File "/usr/lib/python3.5/json/__init__.py", line 319, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.5/json/decoder.py", line 339, in decode
    obj, end = s

In [ ]:
def scrape(start_date, end_date, proxy, keyword="Bitcoin"):
    
    
    
    for dt in rrule(MONTHLY, dtstart=start_date, until=end_date):
        
        fname = "{}/extracted/{}.csv".format(keyword.lower(), dt.strftime('%Y-%m-%d'))
        current_directory = os.getcwd()
        final_directory = os.path.join(current_directory, fname)                                     
        
        #do if file dosen't exisst
        if not os.path.exists(final_directory):           
            df = pd.DataFrame(columns=['ID', 'Tweet', 'Time', 'User', 'Likes', 'Replies', 'Retweet', 'in_response_to', 'response_type'])

            begin = dt.date() - relativedelta(months=1) - relativedelta(days=1)
            end = dt.date()

            print("{} {}".format(begin, end))

            selectedDays = (end - begin).days

            list_of_tweets = query_tweets(keyword, 10000 * selectedDays, begindate=begin, enddate=end, poolsize=selectedDays, proxies=proxy)

            for tweet in list_of_tweets:
                res_type = tweet.response_type
                
                if (tweet.reply_to_id == '0'):
                    res_type='tweet'
                    
                df = df.append({'ID': tweet.id, 'Tweet': tweet.text, 'Time': tweet.timestamp, 'User': tweet.user, 'Likes': tweet.likes, 'Replies': tweet.replies, 'Retweet': tweet.retweets, 'in_response_to': tweet.reply_to_id, 'response_type': tweet.response_type}, ignore_index=True)

            df.to_csv(fname, index=False)

In [140]:
detailsList = []

today = date(datetime.now().year, datetime.now().month, datetime.now().day)

coinDetails = {}
coinDetails['name'] = 'Bitcoin'
coinDetails['start'] = date(2015, 1, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dash'
coinDetails['start'] = date(2015, 5, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dogecoin'
coinDetails['start'] = date(2015, 1, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ethereum'
coinDetails['start'] = date(2015, 12, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Litecoin'
coinDetails['start'] = date(2015, 1, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Monero'
coinDetails['start'] = date(2015, 1, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ripple'
coinDetails['start'] = date(2015, 1, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Stellar'
coinDetails['start'] = date(2015, 1, 1)
coinDetails['end'] = today
detailsList.append(coinDetails)

In [136]:
def fix_paths():
    for coinDetail in detailsList:

        current_directory = os.getcwd()
        final_directory = os.path.join(current_directory, coinDetail['name'].lower())

        if not os.path.exists(final_directory):
            os.makedirs(final_directory)

        if not os.path.exists(final_directory + "/extracted"):
            os.makedirs(final_directory + "/extracted")
            
#fix_paths()

In [137]:
def delete_files():
    for coinDetail in detailsList:
        current_directory = os.getcwd()
        final_directory = os.path.join(current_directory, coinDetail['name'].lower()) + "/extracted/*"
        
        for f in glob(final_directory):
            print("Deleting {}".format(f))
            os.remove(f)
            
            
#delete_files()

In [ ]:
proxies = get_proxies()
proxySize = len(proxies)

for coinDetail in detailsList:
    print("Scraping {} Data".format(coinDetail['name']))
    print("Starting Year: {} Ending Year: {}".format(coinDetail['start'].year, coinDetail['end'].year))
    count = 0
    
    proxy = proxies[count]
    
    for d in range(coinDetail['start'].year, coinDetail['end'].year + 1):
        
        if d == 2018:
            scrape(date(d, 1, 1), coinDetail['end'], proxy=proxy, keyword=coinDetail['name'])
        else:
            if count == 0:
                scrape(coinDetail['start'], date(d, 12, 31), proxy=proxy, keyword=coinDetail['name'])
            else:
                scrape(date(d, 1, 1), date(d, 12, 31), proxy=proxy, keyword=coinDetail['name'])
            
        count +=1
        
        if (count >= proxySize):
            count = 0

In [145]:
import numpy as np

In [146]:
%matplotlib inline

In [213]:
def get_paths():
    dirs = glob("{}/*/".format(os.getcwd()))
    validPath = []

    for directory in dirs:
        fullPath = directory + "extracted"

        if (os.path.exists(fullPath)):
            validPath.append(fullPath)
            
    return validPath

In [235]:
def clean_data(fullDf):
    fullDf = fullDf.reset_index(drop=True)
    fullDf = fullDf.replace(np.inf, np.nan)
    fullDf = fullDf.fillna(method='ffill')

    fullDf['Time'] = pd.to_datetime(fullDf['Time'], errors='coerce')
    fullDf = fullDf.dropna(subset=['Time'])
    fullDf = fullDf.sort_values(by='Time')
    fullDf = fullDf.drop_duplicates()
    
    return fullDf

In [210]:
def merge_data():
    for fullPath in get_paths():
        coinName = fullPath.split("/")[-2]
        print(coinName)

        fullDf = pd.DataFrame(columns=['ID', 'Tweet', 'Time', 'User', 'Likes', 'Replies', 'Retweet', 'in_response_to'])
        writeFile = fullPath + "/" + "combined.csv"

        if (os.path.isfile(writeFile)):
            os.remove(writeFile)

        for file in os.listdir(fullPath):
            csv = fullPath + "/" + file

            if (".csv" in csv):
                df = pd.read_csv(csv, engine='python')
                print(file)
                fullDf = pd.concat([fullDf, df[['ID', 'Tweet', 'Time', 'User', 'Likes', 'Replies', 'Retweet', 'in_response_to']]])
        
        fullDf = clean_data(fullDf)
        fullDf.to_csv(writeFile, index=False)

In [158]:
merge_data()

ripple
2017-11-01.csv
2015-09-01.csv
2016-03-01.csv
2015-11-01.csv
2016-02-01.csv
2016-05-01.csv
2016-09-01.csv
2016-04-01.csv
2016-12-01.csv
2017-01-01.csv
2018-04-01.csv
2016-11-01.csv
2017-09-01.csv
2015-08-01.csv
2018-03-01.csv
2017-06-01.csv
2018-02-01.csv
2018-01-01.csv
2015-05-01.csv
2017-03-01.csv
2018-05-01.csv
2017-02-01.csv
2016-10-01.csv
2016-01-01.csv
2017-12-01.csv
2017-04-01.csv
2017-07-01.csv
2018-06-01.csv
2017-05-01.csv
2015-10-01.csv
2015-02-01.csv
2015-03-01.csv
2017-10-01.csv
2015-06-01.csv
2017-08-01.csv
2015-01-01.csv
2015-04-01.csv
2016-07-01.csv
2016-06-01.csv
2016-08-01.csv
2015-12-01.csv
2015-07-01.csv
dogecoin
2017-11-01.csv
2015-09-01.csv
2016-03-01.csv
2015-11-01.csv
2016-02-01.csv
2016-05-01.csv
2016-09-01.csv
2016-04-01.csv
2016-12-01.csv
2017-01-01.csv
2018-04-01.csv
2016-11-01.csv
2017-09-01.csv
2015-08-01.csv
2018-03-01.csv
2017-06-01.csv
2018-02-01.csv
2018-01-01.csv
2015-05-01.csv
2017-03-01.csv
2018-05-01.csv
2017-02-01.csv
2016-10-01.csv
2016-01-0

In [252]:
def get_missing_days(df, freq=24):
    df['Time'] = pd.to_datetime(df['Time'])

    new = pd.DataFrame(columns=['Time', 'Frequency'])

    new['Time'] = df['Time']
    new['Frequency'] = 1

    new.set_index('Time', inplace = True)
    per_24 = new['Frequency'].resample('{}H'.format(freq)).sum().fillna(0)
    miss_dates = per_24[per_24 == 0].index
    return miss_dates
        
def fill_missing_days():
    for fullPath in get_paths():
        coinName = fullPath.split("/")[-2]
        print(coinName)
        
        df = pd.read_csv('{}/combined.csv'.format(fullPath))
        miss_dates = get_missing_days(miss_dates)
        
        tweetDf = pd.DataFrame(columns=['ID', 'Tweet', 'Time', 'User', 'Likes', 'Replies', 'Retweet', 'in_response_to', 'response_type'])
        
        for dates in miss_dates:
            date_from = dates - pd.DateOffset(days=2) #might wanna make it 2
            date_to = dates + pd.DateOffset(days=2)

            print("{} to {}".format(date_from.date(), date_to.date()))

            list_of_tweets = query_tweets(coinName, 10000, begindate=date_from.date(), enddate=date_to.date(), poolsize=4)

            for tweet in list_of_tweets:
                tweetDf = tweetDf.append({'ID': tweet.id, 'Tweet': tweet.text, 'Time': tweet.timestamp, 'User': tweet.user, 'Likes': tweet.likes, 'Replies': tweet.replies, 'Retweet': tweet.retweets, 'in_response_to': tweet.reply_to_id. 'response_type': tweet.response_type}, ignore_index=True)
                
        tweetDf.to_csv("{}/missing.csv".format(fullPath), index=False)

In [221]:
fill_missing_days()

ripple
2015-06-15 to 2015-06-19


INFO:root:Querying ripple since:2015-06-16 until:2015-06-17
INFO:root:Querying ripple since:2015-06-18 until:2015-06-19
INFO:root:Querying ripple since:2015-06-15 until:2015-06-16
INFO:root:Querying ripple since:2015-06-17 until:2015-06-18
INFO:root:Got 55 tweets for ripple%20since%3A2015-06-16%20until%3A2015-06-17.
INFO:root:Got 55 tweets (55 new).
INFO:root:Got 48 tweets for ripple%20since%3A2015-06-18%20until%3A2015-06-19.
INFO:root:Got 103 tweets (48 new).
INFO:root:Got 60 tweets for ripple%20since%3A2015-06-15%20until%3A2015-06-16.
INFO:root:Got 163 tweets (60 new).
INFO:root:Got 61 tweets for ripple%20since%3A2015-06-17%20until%3A2015-06-18.
INFO:root:Got 224 tweets (61 new).


2016-01-24 to 2016-01-28


INFO:root:Querying ripple since:2016-01-25 until:2016-01-26
INFO:root:Querying ripple since:2016-01-26 until:2016-01-27
INFO:root:Querying ripple since:2016-01-24 until:2016-01-25
INFO:root:Querying ripple since:2016-01-27 until:2016-01-28
INFO:root:Got 45 tweets for ripple%20since%3A2016-01-25%20until%3A2016-01-26.
INFO:root:Got 45 tweets (45 new).
INFO:root:Got 69 tweets for ripple%20since%3A2016-01-26%20until%3A2016-01-27.
INFO:root:Got 114 tweets (69 new).
INFO:root:Got 61 tweets for ripple%20since%3A2016-01-27%20until%3A2016-01-28.
INFO:root:Got 175 tweets (61 new).
INFO:root:Got 60 tweets for ripple%20since%3A2016-01-24%20until%3A2016-01-25.
INFO:root:Got 235 tweets (60 new).


2016-02-04 to 2016-02-08


...

INFO:root:Querying bitcoin since:2017-06-24 until:2017-06-25
INFO:root:Querying bitcoin since:2017-06-25 until:2017-06-26
INFO:root:Querying bitcoin since:2017-06-23 until:2017-06-24
INFO:root:Querying bitcoin since:2017-06-26 until:2017-06-27
INFO:root:Got 261 tweets for bitcoin%20since%3A2017-06-24%20until%3A2017-06-25.
INFO:root:Got 261 tweets (261 new).
INFO:root:Got 255 tweets for bitcoin%20since%3A2017-06-25%20until%3A2017-06-26.
INFO:root:Got 516 tweets (255 new).
INFO:root:Got 316 tweets for bitcoin%20since%3A2017-06-23%20until%3A2017-06-24.
INFO:root:Got 832 tweets (316 new).
INFO:root:Got 343 tweets for bitcoin%20since%3A2017-06-26%20until%3A2017-06-27.
INFO:root:Got 1175 tweets (343 new).


2017-06-24 to 2017-06-28


INFO:root:Querying bitcoin since:2017-06-24 until:2017-06-25
INFO:root:Querying bitcoin since:2017-06-26 until:2017-06-27
INFO:root:Querying bitcoin since:2017-06-27 until:2017-06-28
INFO:root:Querying bitcoin since:2017-06-25 until:2017-06-26
INFO:root:Got 0 tweets for bitcoin%20since%3A2017-06-25%20until%3A2017-06-26.
INFO:root:Got 0 tweets (0 new).
INFO:root:Got 261 tweets for bitcoin%20since%3A2017-06-24%20until%3A2017-06-25.
INFO:root:Got 261 tweets (261 new).
INFO:root:Got 343 tweets for bitcoin%20since%3A2017-06-26%20until%3A2017-06-27.
INFO:root:Got 604 tweets (343 new).
INFO:root:Got 352 tweets for bitcoin%20since%3A2017-06-27%20until%3A2017-06-28.
INFO:root:Got 956 tweets (352 new).


2017-07-19 to 2017-07-23


INFO:root:Querying bitcoin since:2017-07-20 until:2017-07-21
INFO:root:Querying bitcoin since:2017-07-19 until:2017-07-20
INFO:root:Querying bitcoin since:2017-07-21 until:2017-07-22
INFO:root:Querying bitcoin since:2017-07-22 until:2017-07-23
ERROR:root:Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-888399320982183936-888532542567071744-BD1UO2FFu9QAAAAAAAAVfAAAAAcAAABWAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [247]:
def merge_missing():
    for fullPath in get_paths():
        coinName = fullPath.split("/")[-2]
        
        df = pd.read_csv('{}/combined.csv'.format(fullPath), engine='python')
        missing = pd.read_csv('{}/missing.csv'.format(fullPath), engine='python')[['ID', 'Tweet', 'Time', 'User', 'Likes', 'Replies', 'Retweet', 'in_response_to']]
        
        combined = pd.concat([df,missing])
        newDf = clean_data(combined)
        
        newDf.to_csv('{}/combined.csv'.format(fullPath), index=False)
        print("New Data Written to combined.csv")
        os.remove('{}/missing.csv'.format(fullPath))
        print("missing.csv removed")

In [248]:
merge_missing()

New Data Written to combined.csv
missing.csv removed
New Data Written to combined.csv
missing.csv removed
New Data Written to combined.csv
missing.csv removed
New Data Written to combined.csv
missing.csv removed
New Data Written to combined.csv
missing.csv removed
New Data Written to combined.csv
missing.csv removed
New Data Written to combined.csv
missing.csv removed
New Data Written to combined.csv
missing.csv removed


In [253]:
def print_missing(freq=24):
    for fullPath in get_paths():
        coinName = fullPath.split("/")[-2]
        
        df = pd.read_csv('{}/combined.csv'.format(fullPath), engine='python')
        coinName = fullPath.split("/")[-2]
        
        print(coinName)
        print(get_missing_days(df, freq=freq))

In [254]:
print_missing(freq=12)

ripple
DatetimeIndex(['2015-01-01', '2016-02-02', '2016-02-29'], dtype='datetime64[ns]', name='Time', freq=None)
dogecoin
DatetimeIndex([], dtype='datetime64[ns]', name='Time', freq='12H')
monero
DatetimeIndex(['2017-02-13'], dtype='datetime64[ns]', name='Time', freq='12H')
stellar
DatetimeIndex([], dtype='datetime64[ns]', name='Time', freq='12H')
litecoin
DatetimeIndex([], dtype='datetime64[ns]', name='Time', freq='12H')
ethereum
DatetimeIndex(['2018-05-06 00:00:00', '2018-05-06 12:00:00'], dtype='datetime64[ns]', name='Time', freq='12H')
dash
DatetimeIndex([], dtype='datetime64[ns]', name='Time', freq='12H')
bitcoin
DatetimeIndex([], dtype='datetime64[ns]', name='Time', freq='12H')
